# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
#Import cities file as DataFrame
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rosarito,32.3333,-117.0333,57.20,44,40,9.22,MX,1611770794
1,1,Ereymentau,51.6199,73.1033,21.70,98,88,17.05,KZ,1611770876
2,2,Punta Arenas,-53.1500,-70.9167,50.00,50,75,26.46,CL,1611770810
3,3,Mataura,-46.1927,168.8643,48.99,87,62,15.01,NZ,1611770700
4,4,Upernavik,72.7868,-56.1549,-0.87,91,100,3.09,GL,1611770707


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#configure gmaps
gmaps.configure(api_key = g_key)

In [18]:
#Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

#Store humidity
humidity = cities_df['Humidity']

In [19]:
#Plot Heatmap
fig = gmaps.figure()

#Add Heatmap layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10, 
                                 point_radius=1 )
#Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
weather_df = cities_df[(cities_df['Max Temp'] > 70)]
weather_df = weather_df[(weather_df['Max Temp'] < 80)]
weather_df = weather_df[(weather_df['Wind Speed'] < 10)]
weather_df = weather_df[(weather_df['Cloudiness'] == 0)]


count = len(weather_df['Max Temp'])
print(count)

weather_df.head(10)


8


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
119,119,Negombo,7.2083,79.8358,77.00,73,0,9.22,LK,1611770892
327,327,São Filipe,14.8961,-24.4956,76.93,63,0,5.59,CV,1611770920
358,358,Salalah,17.0151,54.0924,75.20,69,0,3.44,OM,1611770924
368,368,Tiznit Province,29.5833,-9.5000,70.39,33,0,3.60,MA,1611770926
449,449,Chopda,21.2500,75.3000,70.45,32,0,5.48,IN,1611770937
487,487,Tombouctou,20.0000,-3.0000,76.87,16,0,9.31,ML,1611770945
489,489,Valparaíso,-33.0393,-71.6273,77.00,44,0,6.91,CL,1611770778
494,494,Araouane,18.9048,-3.5265,76.69,16,0,8.14,ML,1611770946


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:

target_radius = 5000
target_type = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

coordinates = ['{}, {}'.format(lat, lng) for lat, lng in zip(weather_df['Lat'], weather_df['Lng'])]
p_requests = [requests.get(base_url, params={
    'location':coordinate,
    'radius':target_radius,
    'type':target_type,
    'key':g_key
}) for coordinate in coordinates]

print(p_requests)




[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


In [23]:
places_data = [place.json() for place in p_requests]
places_data


[{'html_attributions': [],
  'next_page_token': 'ATtYBwJN1ttByhFcGckSrRVBmWbMBjjAq1dbnXCHXXJhRywf9EcQg3A1fdU72fT-e1nh-YHe3wwUtlUsOD93pCp9cmgL4ZNZwpVAA3_5L5oqvTIhOkdhTe08OyeuIxue0Aeil0os7fkBamgogd2IPf7zs8Pp35RSdHBkmf2jMSdZSC-W6ubhe3fbavh9J94ys1uy7O_Jobrl08lXzbLUaXDC08I7gjt3amuxhAHsSWm0E4-3toWCX_rldnU25BdpHXsTZrn8Ky81UUcE-HaHywQ4ldLSLC3SHQg9R3j-2Bb_AgLqJfcfEhAge2Tn4owr_sGZP02DqTPq_5-dGTgoNVfKyl5sXgvRXLUL5FL35bDpmhLhh2K-Z_fqiyTe4MVVhb6RXV68SKS-NNCYEWQRd8kI0jaZtAdLmk-IYPFFSB4WIa2d6vwFOszrtBfz',
  'results': [{'business_status': 'OPERATIONAL',
    'geometry': {'location': {'lat': 7.225423999999999, 'lng': 79.8396971},
     'viewport': {'northeast': {'lat': 7.226711830291502,
       'lng': 79.8410367302915},
      'southwest': {'lat': 7.224013869708497, 'lng': 79.8383387697085}}},
    'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
    'name': 'Hotel Sunset Beach',
    'photos': [{'height': 3024,
      'html_attributions': ['<a href="https://maps.google.

In [24]:
#[len(place['results']) for place in places_data]
#len(place['results']) > 0



In [25]:
[place['results'][0] for place in places_data if len(place['results']) > 0]

hotel_list = [h_attribute['results'][0]['name'] for h_attribute in places_data if len(h_attribute['results']) > 0]
lat_list = [h_attribute['results'][0]['geometry']['location']['lat'] for h_attribute in places_data if len(h_attribute['results']) > 0]
lng_list = [h_attribute['results'][0]['geometry']['location']['lng'] for h_attribute in places_data if len(h_attribute['results']) > 0]
pid_list = [h_attribute['results'][0]['place_id'] for h_attribute in places_data if len(h_attribute['results']) > 0]
lng_list


hotel_df = pd.DataFrame(data={'Hotel Name': hotel_list, 'Lat': lat_list, 'Lng': lng_list,'Place ID': pid_list }, copy=True)

hotel_df['City'] = ""
hotel_df['Country'] = ""

hotel_df


,Hotel Name,Lat,Lng,Place ID,City,Country
0,Hotel Sunset Beach,7.225424,79.839697,ChIJEbmIQKXu4joR_SFARpRAtp4,,
1,Tortuga B&B,14.880736,-24.492962,ChIJCa3OQEIWNgkRP21GdGKRcOM,,
2,HAMDAN PLAZA HOTEL SALALAH,17.019113,54.061124,ChIJfcmTanVg0T0RVP-fvHfooVk,,
3,पवन टेलर,21.247577,75.300480,ChIJBSM-LfDf2DsRHbIUAi1Z7C0,,
4,Hotel Ultramar,-33.048771,-71.632431,ChIJm6v8gC_hiZYRC-V8uVOkMbw,,


In [26]:
d_base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
a_c = 'address_component'

params = {
    'fields': a_c,
    'key':g_key
}

for index, row in hotel_df.iterrows():
    p_id  = row['Place ID']
    params['place_id'] = p_id
    
    d_requests = requests.get(d_base_url, params=params).json()
    
    itms = d_requests['result']['address_components']
    locality = None
    for itm in itms:
        if 'locality' in itm['types']:
            locality = itm['long_name']
            
            hotel_df.loc[index, 'City'] = locality
            
           
            
    itms = d_requests['result']['address_components']
    country = None
    for itm in itms:
        if 'country' in itm['types']:
            country = itm['long_name']
            
            hotel_df.loc[index, 'Country'] = country
            
               
            
hotel_df


,Hotel Name,Lat,Lng,Place ID,City,Country
0,Hotel Sunset Beach,7.225424,79.839697,ChIJEbmIQKXu4joR_SFARpRAtp4,Negombo,Sri Lanka
1,Tortuga B&B,14.880736,-24.492962,ChIJCa3OQEIWNgkRP21GdGKRcOM,São Filipe,Cape Verde
2,HAMDAN PLAZA HOTEL SALALAH,17.019113,54.061124,ChIJfcmTanVg0T0RVP-fvHfooVk,Salalah,Oman
3,पवन टेलर,21.247577,75.300480,ChIJBSM-LfDf2DsRHbIUAi1Z7C0,Chopda,India
4,Hotel Ultramar,-33.048771,-71.632431,ChIJm6v8gC_hiZYRC-V8uVOkMbw,Valparaíso,Chile


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))